In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Import Library

In [ ]:
import numpy as np
import pandas as pd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set()

# sklearn models & tools
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer
# from sklearn.mixture import GaussianMixture
# from sklearn.preprocessing import RobustScaler
# from sklearn.decomposition import PCA

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

import time

## Load Data

In [ ]:
%%time
submission = pd.read_csv('/kaggle/input/santander-customer-transaction-prediction/sample_submission.csv')
train = pd.read_csv('/kaggle/input/santander-customer-transaction-prediction/train.csv')
test = pd.read_csv('/kaggle/input/santander-customer-transaction-prediction/test.csv')

In [ ]:
# check data
train.head()

In [ ]:
test.head()

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
train.info()

In [ ]:
test.info()

#### train
- ID_code : (object) string data type
- target : int => target variable
- var_0 ~ var_199 : (float) 200 numerical variables  

#### test
- ID_code : (object) string data type
- var_0 ~ var_199 : (float) 200 numerical variables

#### Except for ID_code, all are numerical variables.

### Check null data

In [ ]:
train.isnull().sum().sum()

In [ ]:
test.isnull().sum().sum()

There is no missing data in both train and test datasets.

## EDA
- 데이터 파악을 위해 데이터를 샅샅이 탐색해보도록 하겠습니다.
- We will explore the data to understand the data.

- 모델링의 목적에 맞는 EDA를 진행해서 어떤 식으로 모델링을 해야 할 지 정해보도록 하겠습니다.

### 모델링의 목적 = 분석 목적
- Purpose of modeling = Purpose of analysis

- train dataset의 익명의 수치형 데이터들을 통해 test data의 target이 0일지 1일지 예측하는 분류문제
- Classification problem predicting whether the target of test dataset is 0 or 1 through anonymous numerical data of train dataset.
 
 
- 중요한 포인트는 피처의 정보는 오직 수치로만 확인 가능하다는 점이다. 직관적으로 피처 간의 관계를 파악하기 힘들다.
- The important point is that feature information can only be checked with numbers. It is difficult to intuitively grasp the relationship between features.


- 이러한 데이터셋과 분석 목적을 가질 때 어떤 모델링을 해야 할까? 그리고 이 모델링의 목적에 맞는 EDA는 어떻게 진행하면 좋을까?
- What modeling should we do when we have these datasets and analysis purposes? And how should we proceed with the EDA that fits the purpose of this modeling?


- **지수님이 설명해주신 lightgbm 동작원리와 관련해서 설명을 적으면 좋을 것 같습니다.**

### target data 

In [ ]:
sns.countplot(train.target)

In [ ]:
train.loc[train.target==1].shape[0] / train.loc[train.target==0].shape[0]

In [ ]:
train["target"].value_counts()

#### class imbalanced

- We have to solve the problem about imbalanced class.
- 거래를 할 고객이 거래를 하지 않을 고객보다 훨씩 적다.
- There are far fewer customers who will transaction than those who will not.

In [ ]:
train.describe()

In [ ]:
test.describe()

- train, test 데이터의 집계값이 비슷해 보인다.
- The aggregated values of train and test data look similar.

- 피처 간 집계값들은 조금 차이가 있어 보인다.
- There seems to be a difference in the aggregated values between features.

#### feature correlation

In [ ]:
(train.drop(["target", "ID_code"], axis=1).corr()).mean().mean()

In [ ]:
train.drop(["target", "ID_code"], axis=1).corr()

In [ ]:
# 코드 그대로 가져옴
train_correlations = train.drop(["target"], axis=1).corr()
train_correlations = train_correlations.values.flatten()
train_correlations = train_correlations[train_correlations != 1]

test_correlations = test.corr()
test_correlations = test_correlations.values.flatten()
test_correlations = test_correlations[test_correlations != 1]

plt.figure(figsize=(20,5))
sns.distplot(train_correlations, color="Red", label="train")
sns.distplot(test_correlations, color="Green", label="test")
plt.xlabel("Correlation values found in train (except 1)")
plt.ylabel("Density")
plt.title("Are there correlations between features?"); 
plt.legend()

모든 변수가 선형 상관관계를 가지지 않는 것으로 보인다.

#### 중요 변수 뽑기

EDA를 통해 피처들의 히스토그램을 살펴볼 때 200개의 모든 변수를 확인하는 것보다 몇 개를 뽑아서 확인하는 것이 더 효율적이라고 생각.

- 그냥 뽑는 것보다 중요 변수 선택 방법을 이용해서 뽑는 것이 더 효과적이라고 생각.
- 모든 변수들이 선형 상관관계를 가지지 않기 때문에 nonlinear model을 사용해서 중요 변수를 뽑아보자.
- random forest를 사용하고, 모델의 내장함수인 feature_importances_를 사용해서 중요 변수를 뽑을 것이다.

In [ ]:
%%time
parameters = {'min_samples_leaf': [20, 25]}
forest = RandomForestClassifier(max_depth=15, n_estimators=15)
grid = GridSearchCV(forest, parameters, cv=3, n_jobs=-1, verbose=2, scoring=make_scorer(roc_auc_score))

In [ ]:
# 모델 학습
grid.fit(train.drop(["target", "ID_code"], axis=1).values, train.target.values)

In [ ]:
grid.best_params_

In [ ]:
grid.best_score_

In [ ]:
# 모델의 내장 함수인 feature_importances_
grid.best_estimator_.feature_importances_ 

# 어레이 형태로 반환 # shape : 200

In [ ]:
n_top = 5

# 변수 중요도
importances = grid.best_estimator_.feature_importances_

# 변수 중요도 상위 5개의 인덱스
idx = np.argsort(importances)[::-1][0:n_top] 

# 변수 이름
feature_names = train.drop(["target", "ID_code"], axis=1).columns.values 

# 변수 중요도 기준 상위 5개 변수의 중요도 시각화
plt.figure(figsize=(20,5))
sns.barplot(x=feature_names[idx], y=importances[idx])
plt.title("What are the top important features?")

Q. 중요 변수를 몇개를 뽑아서 탐색하는 것이 전체 EDA에 어떤 도움이 되는지?

In [ ]:
fig, ax = plt.subplots(n_top, 2, figsize=(20, 5*n_top))

for n in range(n_top):
    sns.distplot(train.loc[train.target==0, feature_names[idx][n]], ax=ax[n,0], color="Orange", norm_hist=True)
    sns.distplot(train.loc[train.target==1, feature_names[idx][n]], ax=ax[n,0], color="Red", norm_hist=True)
    sns.distplot(test.loc[:, feature_names[idx][n]], ax=ax[n,1], color="Mediumseagreen", norm_hist=True)
    ax[n,0].set_title("Train {}".format(feature_names[idx][n]))
    ax[n,1].set_title("Test {}".format(feature_names[idx][n]))
    ax[n,0].set_xlabel("")
    ax[n,1].set_xlabel("")

- describe() 함수에서도 확인했듯이, train과 test의 분포가 비슷하다
- train에서 target=0, 1의 pdf가 대체로 비슷한데, 완전 다르게 target=1에서 갑자기 솟아오른 지점이 있다.(갑자기 누적된 부분)


- significant different distribution for the two target values.
- target=1과 target=0의 분포 모양이 달라야 확실하게 구분 가능하고, 이러한 변수를 사용해야 예측 정확도를 높일 수 있다.
- 선택된 변수들은 target에 따라 데이터의 분포가 다른 변수들이다.

- train의 target=1인 데이터들은 

- train, test의 분포는 완전 비슷함

In [ ]:
top = train.loc[:, feature_names[idx]]
top.describe()

In [ ]:
# scatter plot
top = top.join(train.target)
sns.pairplot(top, hue="target")

- target=1인 데이터들이 갑자기 누적되어서 거의 넘어가지 않는 날카로운 지점이 있다.
- 예를 들어, 81번 변수에서는 10에, 12번 변수에서는 13.5에 데이터가 누적되어 날카롭게 솟은 지점이 있다.(limit)

In [ ]:
top

In [ ]:
train.columns.values[2:202]

로우별 평균값의 분포 확인(train, test)

In [ ]:
plt.figure(figsize=(16,6))
features = train.columns.values[2:202]
plt.title("Distribution of mean values per row in the train and test set")
sns.distplot(train[features].mean(axis=1), color="green", kde=True, bins=120, label='train')
sns.distplot(test[features].mean(axis=1), color="blue", kde=True, bins=120, label='test')
plt.legend()
plt.show()

- 여기는 집계값 확인을 위한 EDA인데, 넣을지 말지 고민됨..

## feature engineering

- 기존 변수를 기반으로 새로운 변수 생성
- 기존 변수의 특성을 반영하는(가지고 있는) 변수 생성
- target=1, 0을 구분하는데 더욱 도움이 되는 변수 


### 1)
#### 반올림 및 분위수 기반 binning : label encoding

1. 반올림 : np.round
    - 그냥 반올림한 것, 반올림x10, 반올림x100


2. 분위수 기반 binning : pd.qcut
    - 앞서 뽑은 5개의 중요 변수의 데이터(수치형)의 정확한 값 말고 데이터들의 범위(범주)를 알기 위해 (values range than actual values)
    - qcut함수를 이용해서 분위수 기반의 bin을 만들고, LabelEncoder로 라벨링하여 새로운 변수 생성
    - We will use qcut to create 10 equally sized bins i.e quartiles
    - qcut function can be used to generate equally sized quantiles bins for your data

===============================================================================

- Histograms are example of data binning that helps to visualize your data distribution in equal intervals

- qcut : bin을 만들기 위한 분위수 기반의 함수
- qcut is a quantile based function to create bins

- Quantile is to divide the data into equal number of subgroups or probability distributions of equal probability into continuous interval
- Quantile은 데이터를 동일한 수의 subgroups으로 나누거나 같은 확률의 확률 분포를 연속 구간으로 나누는 것입니다.

In [ ]:
%%time
original_features = train.drop(["target", "ID_code"], axis=1).columns.values

# 5개의 중요 변수 qcut한 변수 생성
encoder = LabelEncoder()
for your_feature in top.drop("target", axis=1).columns.values:
    train[your_feature + "_qbinned"] = pd.qcut(
        train.loc[:, your_feature].values,
        q=10,
        labels=False
    )
    train[your_feature + "_qbinned"] = encoder.fit_transform(
        train[your_feature + "_qbinned"].values.reshape(-1, 1)
    )
    
# 5개의 중요 변수 반올림한 변수 생성
encoder = LabelEncoder()
for your_feature in top.drop("target", axis=1).columns.values:
    train[your_feature + "_rounded"] = np.round(train.loc[:, your_feature].values)
    train[your_feature + "_rounded_10"] = np.round(10*train.loc[:, your_feature].values)
    train[your_feature + "_rounded_100"] = np.round(100*train.loc[:, your_feature].values)

    
# test에도 같이 적용

encoder = LabelEncoder()
for your_feature in top.drop("target", axis=1).columns.values:
    test[your_feature + "_qbinned"] = pd.qcut(
        test.loc[:, your_feature].values,
        q=10,
        labels=False
    )
    test[your_feature + "_qbinned"] = encoder.fit_transform(
        test[your_feature + "_qbinned"].values.reshape(-1, 1)
    )

encoder = LabelEncoder()
for your_feature in top.drop("target", axis=1).columns.values:
    test[your_feature + "_rounded"] = np.round(test.loc[:, your_feature].values)
    test[your_feature + "_rounded_10"] = np.round(10*test.loc[:, your_feature].values)
    test[your_feature + "_rounded_100"] = np.round(100*test.loc[:, your_feature].values)

In [ ]:
train.head()

In [ ]:
train.columns.values[2:202]

### 2)
#### row별 집계값 : sum, min, max 등 
- 기존 피처에 대해 몇 개의 집계값을 계산해서 새로운 피처로 추가
- 왜?

In [ ]:
%%time
idx = features = train.columns.values[2:202] # 원래 피처(var_0 ~ var_199)
for df in [test, train]:
    df['sum'] = df[idx].sum(axis=1)  
    df['min'] = df[idx].min(axis=1)
    df['max'] = df[idx].max(axis=1)
    df['mean'] = df[idx].mean(axis=1)
    df['std'] = df[idx].std(axis=1)
    df['skew'] = df[idx].skew(axis=1)
    df['kurt'] = df[idx].kurtosis(axis=1)
    df['med'] = df[idx].median(axis=1)

In [ ]:
# 추가한 변수 확인
train[train.columns[222:]].head()

In [ ]:
test[test.columns[222:]].head()

In [ ]:
train.shape

## Modeling

In [ ]:
features = [c for c in train.columns if c not in ['ID_code', 'target']]
target = train['target']

In [ ]:
# 파라미터
params = {'objective' : "binary",  
               'boost':"gbdt", # gbdt : Gradient Boosting Desicion Tree # 실행하고자 하는 알고리즘 타입 정의
               'metric':"auc", # 성능평가 지표
               'boost_from_average':"false",
               'num_threads':8,
               'learning_rate' : 0.01, # 최종 결과에 대한 각각의 Tree에 영향을 미치는 변수
               'num_leaves' : 13, # 10, 13, 15 # 전체 Tree의 leave 수. Tree 모델의 복잡성을 컨트롤하는 주요 파라미터. # 디폴트 31
               'max_depth':-1,  # tree의 최대 깊이 # 0보다 작은 값은 깊이에 제한이 없음 # 디폴트 -1
               'tree_learner' : "serial",
               'feature_fraction' : 0.05, # 모델이 tree를 만들 때 매번 각각의 iteration에서 파라미터 중 5%를 랜덤하게 선택
               'bagging_freq' : 5,
               'bagging_fraction' : 0.4, # 매번 iteration을 돌 때 사용되는 데이터의 일부를 선택하는데 트레이닝 속도를 높이고 과적합을 방지할 때 주로 사용
               'min_data_in_leaf' : 80, # 70, 80, 90 # Leaf가 가지고 있는 최소한의 레코드 수, 과적합 해결에 사용, 디폴트 20(최적값)
               'min_sum_hessian_in_leaf' : 10.0,
               'verbosity' : 1}

# 일반적으로 n_estimators를 크게 하고 learning_rate를 작게 해서 예측 성능을 향상시킬 수 있으나, 과적합 이슈와 학습시간이 길어지는 부정적인 영향도 고려해야 함

#### Hyper Parameter Tunning
- 성능 높이고 과적합을 줄이는 방향으로 주요 파라미터를 튜닝했다.
- learning_rate : 0.01로 낮을 때 성능이 좋았다.
- num_leaves : 10, 13, 15
- min_data_in_leaf : 70, 80, 90

#### kfold
- class imbalanced 문제를 고려해서 StratifiedKFold 방식을 사용했다.

#### LightGBM
- Dataset이 피처의 수가 많고, 모두 수치형 데이터 타입으로 이루어져 있다.
- 데이터의 수가 LightGBM을 사용하기에 적합하다.(과적합x)

In [ ]:
%%time

folds = StratifiedKFold(n_splits=5, shuffle=False, random_state=44000)
oof = np.zeros(len(train))
predictions = np.zeros(len(test))
feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values, target.values)):
    print("Fold {}".format(fold_))
    trn_data = lgb.Dataset(train.iloc[trn_idx][features], label=target.iloc[trn_idx])
    val_data = lgb.Dataset(train.iloc[val_idx][features], label=target.iloc[val_idx])

    num_round = 1000000 # 다폴트 : 100
    clf = lgb.train(params, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=1000, early_stopping_rounds = 3000)
    oof[val_idx] = clf.predict(train.iloc[val_idx][features], num_iteration=clf.best_iteration)
    # early_stopping_rounds : The index of iteration that has the best performance will be saved in the best_iteration field if early stopping logic is enabled by setting early_stopping_rounds
    # verbose_eval : If int, the eval metric on the valid set is printed at every verbose_eval boosting stage. 
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    predictions += clf.predict(test[features], num_iteration=clf.best_iteration) / folds.n_splits

print("CV score: {:<8.5f}".format(roc_auc_score(target, oof)))